## Extract Weather forcast for Gwinnett county from Nationl Weather Service

In [1]:
import requests

#### Download weather data

In [7]:
r = requests.get("https://forecast.weather.gov/MapClick.php?lat=34.0682&lon=-83.9808#.WuMzm4jwbIU")

In [9]:
r.text[0:509]

'<!DOCTYPE html>\n<html class="no-js">\n    <head>\n        <!-- Meta -->\n        <meta name="viewport" content="width=device-width">\n        <link rel="schema.DC" href="http://purl.org/dc/elements/1.1/" /><title>National Weather Service</title><meta name="DC.title" content="National Weather Service" /><meta name="DC.description" content="NOAA National Weather Service National Weather Service" /><meta name="DC.creator" content="US Department of Commerce, NOAA, National Weather Service" /><meta name="DC.date.'

In [ ]:
from bs4 import BeautifulSoup

In [10]:
soup = BeautifulSoup(r.text,"html.parser")

In [18]:
soup.prettify()

'<!DOCTYPE html>\n<html class="no-js">\n <head>\n  <!-- Meta -->\n  <meta content="width=device-width" name="viewport"/>\n  <link href="http://purl.org/dc/elements/1.1/" rel="schema.DC"/>\n  <title>\n   National Weather Service\n  </title>\n  <meta content="National Weather Service" name="DC.title">\n   <meta content="NOAA National Weather Service National Weather Service" name="DC.description"/>\n   <meta content="US Department of Commerce, NOAA, National Weather Service" name="DC.creator"/>\n   <meta content="" name="DC.date.created" scheme="ISO8601"/>\n   <meta content="EN-US" name="DC.language" scheme="DCTERMS.RFC1766"/>\n   <meta content="weather, National Weather Service" name="DC.keywords"/>\n   <meta content="NOAA\'s National Weather Service" name="DC.publisher"/>\n   <meta content="National Weather Service" name="DC.contributor"/>\n   <meta content="http://www.weather.gov/disclaimer.php" name="DC.rights"/>\n   <meta content="General" name="rating"/>\n   <meta content="index,fo

In [20]:
#list(soup.children)

In [21]:
seven_day = soup.find(id="seven-day-forecast")

In [22]:
seven_day

<div class="panel panel-default" id="seven-day-forecast">
<div class="panel-heading">
<b>Extended Forecast for</b>
<h2 class="panel-title">
	    	    4 Miles SSE Buford GA	</h2>
</div>
<div class="panel-body" id="seven-day-forecast-body">
<div id="seven-day-forecast-container"><ul class="list-unstyled" id="seven-day-forecast-list"><li class="forecast-tombstone">
<div class="tombstone-container">
<p class="period-name">Today<br/><br/></p>
<p><img alt="Today: Mostly sunny, with a high near 72. Northwest wind 10 to 15 mph, with gusts as high as 20 mph. " class="forecast-icon" src="newimages/medium/sct.png" title="Today: Mostly sunny, with a high near 72. Northwest wind 10 to 15 mph, with gusts as high as 20 mph. "/></p><p class="short-desc">Mostly Sunny</p><p class="temp temp-high">High: 72 °F</p></div></li><li class="forecast-tombstone">
<div class="tombstone-container">
<p class="period-name">Tonight<br/><br/></p>
<p><img alt="Tonight: Clear, with a low around 51. West wind 5 to 10 mph,

#### Get tonight's forcast

In [25]:
forcast = seven_day.find_all(class_= "tombstone-container")

### Extract weather information for one day

In [28]:
tonight = forcast[0]

In [29]:
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Mostly sunny, with a high near 72. Northwest wind 10 to 15 mph, with gusts as high as 20 mph. " class="forecast-icon" src="newimages/medium/sct.png" title="Today: Mostly sunny, with a high near 72. Northwest wind 10 to 15 mph, with gusts as high as 20 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Sunny
 </p>
 <p class="temp temp-high">
  High: 72 °F
 </p>
</div>


In [33]:
tonight.find_all('p')[0].text

'Today'

#### Get tonight's short desc and tempeture

In [43]:
Period = tonight.find(class_="period-name").get_text()
Desc = tonight.find(class_="short-desc").get_text()
Temp = tonight.find(class_="temp").get_text()

In [44]:
print(Period)
print(Desc)
print(Temp)

Today
Mostly Sunny
High: 72 °F


#### Get image title for long descriptions

In [51]:
Img = tonight.find('img')
Img['title']

'Today: Mostly sunny, with a high near 72. Northwest wind 10 to 15 mph, with gusts as high as 20 mph. '

### Extract all the informaton at once

In [67]:
Period = []
Days = seven_day.select(".tombstone-container .period-name")
for day in Days:
    Period.append(day.get_text())
    
ShorDesc = []
Desc = seven_day.select(".tombstone-container .short-desc")
for sd in Desc:
    ShorDesc.append(sd.get_text())

Temp = []
Temprature = seven_day.select(".tombstone-container .temp")
for t in Temprature:
    Temp.append(t.get_text())
    
Title = []
Img = seven_day.select(".tombstone-container img")
for i in Img:
    Title.append(i['title'])
    

### Combining a data in to a Pandas Dataframe

In [81]:
import pandas as pd

In [82]:
weather = pd.DataFrame({"Title":Title,"Short Desc":ShorDesc,"Temp":Temp,"Period":Period})

In [83]:
weather

,Period,Short Desc,Temp,Title
0,Today,Mostly Sunny,High: 72 °F,"Today: Mostly sunny, with a high near 72. Nort..."
1,Tonight,Clear,Low: 51 °F,"Tonight: Clear, with a low around 51. West win..."
2,Saturday,Sunny,High: 74 °F,"Saturday: Sunny, with a high near 74. West win..."
3,SaturdayNight,Mostly Clear,Low: 47 °F,"Saturday Night: Mostly clear, with a low aroun..."
4,Sunday,Sunny,High: 69 °F,"Sunday: Sunny, with a high near 69. North wind..."
5,SundayNight,Clear,Low: 45 °F,"Sunday Night: Clear, with a low around 45."
6,Monday,Sunny,High: 74 °F,"Monday: Sunny, with a high near 74."
7,MondayNight,Clear,Low: 51 °F,"Monday Night: Clear, with a low around 51."
8,Tuesday,Sunny,High: 79 °F,"Tuesday: Sunny, with a high near 79."
